# Problem 23

A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.

Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.

---

In [7]:
# reuse the factor code of exercise 21
from functools import reduce
def sum_factors(n):    
    a = set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))
    a.remove(n)
    
    return sum(a)

In [92]:
%%timeit -n1 -r1
# First get all the abudant numbers smaller than N
N = 28123 + 1

abundant_numbers = []
for n in range(1, N):
    if n < sum_factors(n):
        abundant_numbers.append(n)

# Compute the different sums 
import numpy as np
a = np.array(abundant_numbers)
X, Y = np.meshgrid(a, a) # get all permuations 
Z = (X + Y)
Z = Z[Z < 28123 + 1]

Z = list(set(Z.tolist()))

# Eliminate the sums that are possible
options = list(range(1, N))
for z in Z:
    options.remove(z)
print(sum(options))

4179871
2.12 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Bit slow, but it works.

In [161]:
# the next approach was tried but it turns out it is even slower

In [162]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [163]:
%%cython 
#--annotate
import numpy as np
cimport numpy as np
cimport cython

cdef int N, n, ni, nn, abundant_number1, abundant_number2
cdef list abundant_numbers_list

from functools import reduce
def sum_factors(n):    
    a = set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))
    a.remove(n)
    
    return sum(a)

# First get all the abudant numbers smaller than N
N = 28123 + 1

abundant_numbers_list = []
for ni in range(1, N):
    if ni < sum_factors(ni):
        abundant_numbers_list.append(ni)

cdef int[:] abundant_numbers = np.array(abundant_numbers_list)
cdef int[:] vals = np.ones((N,), dtype=np.int32)
for n in range(1, 100): # note we only just do till a 100 for testing 
    for abundant_number1 in abundant_numbers:
        if n > abundant_number1:
            nn = n - abundant_number1
            for abundant_number2 in abundant_numbers:
                if nn == abundant_number2:
                    vals[n-1] = 0
                else:
                    break
        else:
            break